**Análisis exploratorio del texto**

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
import pickle
import nltk
import os
import sklearn.manifold
from collections import Counter
from spacy.tokens import Doc, Token, Span
from nltk.cluster import kmeans, cosine_distance

**A continuación serán procesados para su posterior análisis las capítulos 2 y 12 del libro** *Cien años de soledad*.

**El motivo de selección de dos capítulos es para generar una comparación a medida que avanza la historia**.

In [20]:
cien2 = "capi2.txt"
text_file2 = open(cien2, "r")
dataset2 = text_file2.read()
text_file2.close()

cien12 = "capi12.txt"
text_file12 = open(cien12, "r")
dataset12 = text_file12.read()
text_file12.close()

In [19]:
nlp = spacy.load('es_core_news_sm', vectors=False, entity=False)
doc2 = nlp(dataset2)
doc12 = nlp(dataset12)

In [30]:
# Cuántas oraciones hay en el capítulo 2 y 12?
oraciones2 = [s for s in doc2.sents]
oraciones12 = [s for s in doc12.sents]
print('Cantidad de oraciones en el capítulo 2:')
print(len(oraciones2))

print('Cantidad de oraciones en el capítulo 12:')
print(len(oraciones12))

Cantidad de oraciones en el capítulo 2:
278
Cantidad de oraciones en el capítulo 12:
223


In [31]:
# Mostrar las primeras 10 oraciones para verificar el texto en el capítulo 2
print(oraciones2[1:11])

[Las quemaduras la dejaron convertida en una esposa inútil para toda la vida., No podía sentarse sino de medio lado, acomodada en cojines, y algo extraño debió quedarle en el modo de andar, porque nunca volvió a caminar en público., Renunció a toda clase de hábitos sociales obsesionada por la idea de que su cuerpo despedía un olor a chamusquina., El alba la sorprendía en el patio sin atreverse a dormir, porque soñaba que los ingleses con sus feroces perros de asalto se metían por la ventana del dormitorio y la sometían a vergonzosos tormentos con hierros al rojo vivo., Su marido, un comerciante aragonés con quien tenía dos hijos, se gastó media tienda en medicinas y entretenimientos buscando la manera de aliviar sus terrores., Por último liquidó el negocio y llevó la familia a vivir lejos del mar, en una ranchería de indios pacíficos situada en las estribaciones de la sierra, donde le construyó a su mujer un dormitorio sin ventanas para que no tuvieran por donde entrar los piratas de s

In [32]:
#Queremos ver las primeras dos oraciones del capítulo 2 y como se granfican sus relaciones de dependencia de palabras
from spacy import displacy
displacy.render(oraciones2[1:3], style="dep")

****Ahora observaremos las oraciones en las que aparece 'Buendía' (hasta 5 oraciones) en ambos capítulos****

In [39]:
buendia2 = [sent for sent in doc2.sents if 'Buendía' in sent.string][-5:]
print('Oraciones con palabra Buendía en capítulo 2:')
buendia2

Oraciones con palabra Buendía en capítulo 2:


[Durante varias semanas, José Arcadio Buendía se dejó vencer por la consternación.,
 José Arcadio Buendía y su hijo no supieron en qué momento estaban otra vez en el laboratorio, sacudiendo el polvo, prendiendo fuego al atanor, entregados una vez más a la paciente manipulación de la materia dormida desde hacía varios meses en su cama de estiércol.,
 José Arcadio Buendía y su hijo observaban aquellos fenómenos con asustado alborozo, sin lograr explicárselos, pero interpretándolos como anuncios de la materia.,
 José Arcadio Buendía apenas si pudo resistir el impacto.,
 José Arcadio Buendía tardó mucho tiempo para restablecerse la perplejidad cuando salió a la calle y vio la muchedumbre.]

In [40]:
buendia12 = [sent for sent in doc12.sents if 'Buendía' in sent.string][-5:]
print('Oraciones con palabra Buendía en capítulo 12:')
buendia12

Oraciones con palabra Buendía en capítulo 12:


[Pero el espanto de la desgracia había calado tan hondo, que ni la misma Fernanda se prestó al experimento, y nunca más se vio un Buendía arrodillado en el comulgatorio el miércoles de ceniza.
    ,
 El coronel Aureliano Buendía no logró recobrar la serenidad en mucho tiempo.,
 El coronel Aureliano Buendía era el único habitante de la  casa que no seguía viendo al potente anciano agobiado por medio siglo de intemperie.,
 Después del armisticio de Neerlandia, mientras el coronel Aureliano Buendía se refugiaba en el exilio de sus pescaditos de oro, él se mantuvo en contacto con los oficiales rebeldes que le fueron fieles hasta la derrota.,
 De modo que cuando el coronel Aureliano Buendía lo invitó a promover una conflagración mortal que arrasara con todo vestigio de un régimen de corrupción y de escándalo sostenido por el invasor extranjero, el coronel Gerineldo Márquez no pudo reprimir un estremecimiento de compasión.
    ]

**Queremos conocer los personajes del cuento que aparecen en el capítulo 2 y 12. El comando encontrar personajes nos devolverá una lista de personajes con su cantidad de ocurrencias**

In [41]:
def encontrar_personajes(doc2):
       
    personajes = Counter()
    for ent in doc2.ents:
        if ent.label_ == 'PER':
            personajes[ent.lemma_] += 1
            
    return personajes.most_common()

In [43]:
# Y aquí extraemos los 5 personajes principales del texto y contando cuantas veces son nombrados.
print('Los 5 personajes principales del capítulo 2:')
print(encontrar_personajes(doc2)[:5])

Los 5 personajes principales del capítulo 2:
[('José Arcadio Buendía', 23), ('José Arcadio', 17), ('Aureliano', 9), ('Prudencio Aguilar', 8), ('Pilar Ternera', 4)]


In [44]:
def encontrar_personajes(doc12):
       
    personajes = Counter()
    for ent in doc12.ents:
        if ent.label_ == 'PER':
            personajes[ent.lemma_] += 1
            
    return personajes.most_common()

In [45]:
print('Los 5 personajes principales del capítulo 12:')
print(encontrar_personajes(doc12)[:5])

Los 5 personajes principales del capítulo 12:
[('Aureliano Buendía', 13), ('Herbert', 6), ('Aureliano Segundo', 6), ('Fernanda', 6), ('Amaranta', 5)]


**En el próximo paso identificaremos los adjetivos que corresponden a los personajes**

In [46]:
def obtener_adj_pers(doc2, personaje):
       
    adjetivos = []
    for ent in doc2.ents:
        if ent.lemma_ == personaje:
            for token in ent.subtree:
                if token.pos_ == 'ADJ':
                    adjetivos.append(token.lemma_)
    
    for ent in doc2.ents:
        if ent.lemma_ == personaje:
            if ent.root.dep_ == 'nsubj':
                for child in ent.root.head.children:
                    if child.dep_ == 'acomp':
                        adjetivos.append(child.lemma_)
    
    return adjetivos

In [48]:
# Encontrar adjetivos que describen a algún personaje.
print(obtener_adj_pers(doc2, "José Arcadio Buendía"))

['productivo', 'fastidiar', 'cierto', 'machacar', 'recalentar']


In [52]:
print(obtener_adj_pers(doc2, "Aureliano"))

['\ufeff1', 'raro', 'alquímico', 'pequeño', 'Aureliano', 'misterioso']


**Ahora observaremos las entidades nombradas en el texto**

In [76]:
# Trabajando con las entidades del texto
# Una entidad nombrada es cualquier objeto del mundo real como una persona,
# ubicación, organización o producto con un nombre propio.

# tipos de entidades del texto
set(ent.label_ for ent in doc12.ents)

{'LOC', 'MISC', 'PER'}

In [78]:
# Entidades nombradas de tipo LOC
[ent for ent in doc12.ents if ent.label_ == 'LOC'][:3]

[Macondo, Francia, Macondo]

In [80]:
# Entidades nombradas de tipo PER
[ent for ent in doc12.ents if ent.label_ == 'PER'][:6]

[Aureliano Triste,
 Bruno Crespi,
 Bruno Crespi,
 Ante,
 José Arcadio Buendía,
 Herbert]

In [84]:
# Partes de la oración (POS)
# En las partes de la oración se etiquetan las palabras de acuerdo a lo 
# que significan segun su contexto. Algunas de estas etiquetas pueden
# ser: Adjetivos, verbos, adverbios, conjunciones, pronombres, sustantivos.

# Etiquetas del texto
set(token.pos_ for token in doc12)

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SPACE',
 'SYM',
 'VERB'}

In [85]:
# Etiquetas de tipo ADJ
[token.orth_ for token in doc12 if token.pos_ == 'ADJ'][1:11]

['pálidas',
 'eléctricas',
 'alimentadas',
 'segundo',
 'vivas',
 'comerciante',
 'muerto',
 'sepultado',
 'vivo',
 'convertido']

In [87]:
# Etiquetas de tipo PROPN
[token.orth_ for token in doc2 if token.pos_ == 'PROPN'][1:11]

['Francis',
 'Drake',
 'Riohacha',
 'Úrsula',
 'Iguarán',
 'José',
 'Arcadio',
 'Buendía',
 'Úrsula',
 'Úrsula']

In [88]:
# importando gensim y TSNE para graficar
import gensim
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE

In [89]:
texto = [[str(palabra).lower() for palabra in sent if str(palabra) ]
         for sent in oraciones12]

In [90]:
# generando el diccionario
diccionario = gensim.corpora.Dictionary(texto)

In [97]:
# creando el modelo
modelo = gensim.models.Word2Vec(texto, workers=4, size=10, 
                                min_count=3, window=5, sample=1e-3)

In [99]:
# como convertimos las palabras en vectores y los vectores capturan muchas 
# regularidades lingüísticas, podemos aplicar operaciones vectoriales para
# extraer muchas propiedades interesantes. 

# palabras similares a persona
modelo.most_similar_cosmul('r')

C:\Users\Eugenia\Miniconda3v1\envs\diplodatos\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  


KeyError: "word 'raro' not in vocabulary"